# Introduction

This notebook projects the vehicle count by county in NYS using two data sources:

1. Current vehicle registration data by county (from DMV)
2. Future vehicle count projection in NYS (from NYSERDA Climate Act Scoping Plan)

It assumes that the distribution of vehicle counts by county in NYS will remain the same in the future.

The output of this notebook is a CSV file with the projected vehicle count by county in NYS.

# Packages

In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
# Set up path directory
cwd = os.getcwd()
data_dir = os.path.join(cwd, 'NYSERDA_Data')
output_dir = os.path.join(cwd, 'InputData')
print(f'Data directory: {data_dir}')
print(f'Output directory: {output_dir}')

Data directory: d:\EERL\EV\EVI-Pro-Lite\NYSERDA_Data
Output directory: d:\EERL\EV\EVI-Pro-Lite\InputData


# Current vehicle count

In [3]:
# Read NYS zip code data
zipcode_df = pd.read_csv(os.path.join(data_dir, 'nys_zipcode.csv'))
zipcode_df = zipcode_df[['ZIP Code', 'County']]


In [4]:
# Read current vehicle registration data
vehicle_df = pd.read_csv(os.path.join(data_dir, 'Current Registrations.csv'))

# Drop rows with missing ZIP Code
vehicle_df = vehicle_df[vehicle_df['ZIP Code'] != 'Error']

# Filter for NYS zip codes
vehicle_df['ZIP Code'] = vehicle_df['ZIP Code'].astype(int)
vehicle_df = vehicle_df[vehicle_df['ZIP Code'].isin(zipcode_df['ZIP Code'])]


In [5]:
# Group by zip code and sum vehicle count
vehicle_by_zip = vehicle_df.groupby('ZIP Code')['Vehicle Count'].sum().reset_index()

# Group by county and sum vehicle count
vehicle_by_county = vehicle_by_zip.merge(zipcode_df, on='ZIP Code', how='left')
vehicle_by_county = vehicle_by_county.groupby('County')['Vehicle Count'].sum().reset_index()

# Percentage of vehicles by county
vehicle_by_county['Percentage'] = vehicle_by_county['Vehicle Count'] / vehicle_by_county['Vehicle Count'].sum()
vehicle_by_county

,County,Vehicle Count,Percentage
0,Albany County,236214,0.021072
1,Allegany County,36836,0.003286
2,Bronx County,295553,0.026366
3,Broome County,147283,0.013139
4,Cattaraugus County,60274,0.005377
...,...,...,...
58,Washington County,58142,0.005187
59,Wayne County,87527,0.007808
60,Westchester County,684472,0.061061
61,Wyoming County,35948,0.003207


In [6]:
total_vehicles = vehicle_by_zip['Vehicle Count'].sum()
print(f'Total number of vehicles in NYS: {total_vehicles:.2e}')

Total number of vehicles in NYS: 1.12e+07


# Future vehicle count

In [7]:
# Read NYS total vehicle count projection
# Data source: NYSERDA Climate Act Scoping Plan
# Light duty vehicle (LDV) stock forecast
ldv_proj_df = pd.read_csv(os.path.join(data_dir, 'LDV_stock_forecast.csv'))

# There are five scenarios in the projection
# Reference, AP Recommendations, Strategic Use of Low-Carbon Fuels
# Accelerated Transition Away from Combustion, Beyond 85% Reductions
# We will use the "Reference" scenario
ldv_proj_df = ldv_proj_df[ldv_proj_df['Scenario'] == 'Reference']

# Column names of years from 2020 to 2050
year_cols = ldv_proj_df.columns[2:]

# The unit is million vehicles
ldv_proj_df[year_cols] = ldv_proj_df[year_cols] * 1e6

ldv_proj_df


,Scenario,Vehicle Type,2020,2021,2022,2023,2024,2025,2026,2027,...,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050
0,Reference,Battery Electric,23623.24,35944.49,50554.60,69162.85,96228.17,138156.21,185842.14,247021.08,...,2206796.95,2368262.82,2525069.59,2677670.12,2828524.45,2975148.69,3118470.31,3259772.63,3386770.07,3513044.99
1,Reference,CNG,1676.96,1645.30,1604.99,1556.59,1500.62,1438.13,1369.40,1295.02,...,211.26,170.60,136.87,109.04,86.14,68.00,52.79,40.41,31.63,24.11
2,Reference,Diesel,762658.37,763243.80,763832.25,764423.70,765018.23,765615.77,766216.40,766820.03,...,775596.76,776247.23,776900.86,777557.70,778217.72,778880.96,779547.40,780217.05,780889.95,781566.07
3,Reference,Gasoline,9390397.85,9377303.28,9358894.42,9332055.03,9286905.24,9206796.71,9102059.93,8980554.44,...,6118893.35,5905792.31,5702112.57,5504394.81,5309590.15,5120878.86,4936528.99,4753988.24,4591204.52,4427729.94
4,Reference,Hydrogen Fuel Cell,0.92,0.84,0.75,0.67,0.60,0.52,0.47,0.41,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5,Reference,Plug in Hybrid Electric,20928.99,29306.96,40756.58,56684.31,82510.69,128477.45,193358.71,261559.34,...,1277768.63,1337825.72,1393153.31,1446758.82,1499232.27,1549877.52,1599500.97,1649371.82,1693827.08,1739734.67
6,Reference,Total,10199286.33,10207444.67,10215643.59,10223883.15,10232163.55,10240484.79,10248847.05,10257250.32,...,10379266.95,10388298.68,10397373.20,10406490.49,10415650.73,10424854.03,10434100.46,10443390.15,10452723.25,10462099.78


In [8]:
# Medium and heavy duty vehicle (MHDV) stock forecast
# NOTE:MHDV is not used in the current analysis
mhdv_proj_df = pd.read_csv(os.path.join(data_dir, 'MHDV_stock_forecast.csv'))
mhdv_proj_df = mhdv_proj_df[mhdv_proj_df['Scenario'] == 'Reference']

# The unit is thousand vehicles
mhdv_proj_df[year_cols] = mhdv_proj_df[year_cols] * 1e3
mhdv_proj_df

,Scenario,Vehicle type,2020,2021,2022,2023,2024,2025,2026,2027,...,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050
0,Reference,Battery Electric,1.60,1.50,1.40,1.31,124.40,361.25,712.38,1178.51,...,18900.45,20931.06,23049.47,25243.64,27499.93,29804.07,32142.38,34504.04,36878.15,39257.42
1,Reference,CNG,820.13,993.05,1157.05,1316.11,1472.87,1628.83,1784.48,1939.54,...,3454.62,3489.39,3516.04,3535.91,3550.32,3560.62,3568.00,3573.43,3577.67,3581.27
2,Reference,Diesel,149185.32,149107.86,149039.87,148977.36,148794.55,148499.37,148090.75,147568.24,...,129774.33,127816.41,125779.40,123674.00,121512.53,119307.96,117072.69,114816.60,112549.89,110279.25
3,Reference,Diesel Electric Hybrid,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,Reference,Gasoline,78723.49,78781.34,78839.49,78897.94,78956.68,79015.71,79075.03,79134.66,...,80000.55,80064.65,80129.06,80193.79,80258.82,80324.15,80389.80,80455.75,80522.01,80588.58
5,Reference,Hydrogen Fuel Cell,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
6,Reference,Total,228730.54,228883.75,229037.81,229192.72,229348.50,229505.16,229662.64,229820.95,...,232129.95,232301.51,232473.97,232647.34,232821.60,232996.80,233172.87,233349.82,233527.72,233706.52


In [9]:
# Total amount of vehicle including LDV and MHDV
ldv_mhdv_proj_df = ldv_proj_df.copy()
ldv_mhdv_proj_df[year_cols] = ldv_proj_df[year_cols] + mhdv_proj_df[year_cols]
ldv_mhdv_proj_df

,Scenario,Vehicle Type,2020,2021,2022,2023,2024,2025,2026,2027,...,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050
0,Reference,Battery Electric,23624.84,35945.99,50556.00,69164.16,96352.57,138517.46,186554.52,248199.59,...,2225697.40,2389193.88,2548119.06,2702913.76,2856024.38,3004952.76,3150612.69,3294276.67,3423648.22,3552302.41
1,Reference,CNG,2497.09,2638.35,2762.04,2872.70,2973.49,3066.96,3153.88,3234.56,...,3665.88,3659.99,3652.91,3644.95,3636.46,3628.62,3620.79,3613.84,3609.30,3605.38
2,Reference,Diesel,911843.69,912351.66,912872.12,913401.06,913812.78,914115.14,914307.15,914388.27,...,905371.09,904063.64,902680.26,901231.70,899730.25,898188.92,896620.09,895033.65,893439.84,891845.32
3,Reference,Gasoline,9390397.85,9377303.28,9358894.42,9332055.03,9286905.24,9206796.71,9102059.93,8980554.44,...,6118893.35,5905792.31,5702112.57,5504394.81,5309590.15,5120878.86,4936528.99,4753988.24,4591204.52,4427729.94
4,Reference,Hydrogen Fuel Cell,78724.41,78782.18,78840.24,78898.61,78957.28,79016.23,79075.50,79135.07,...,80000.55,80064.65,80129.06,80193.79,80258.82,80324.15,80389.80,80455.75,80522.01,80588.58
5,Reference,Plug in Hybrid Electric,20928.99,29306.96,40756.58,56684.31,82510.69,128477.45,193358.71,261559.34,...,1277768.63,1337825.72,1393153.31,1446758.82,1499232.27,1549877.52,1599500.97,1649371.82,1693827.08,1739734.67
6,Reference,Total,10428016.87,10436328.42,10444681.40,10453075.87,10461512.05,10469989.95,10478509.69,10487071.27,...,10611396.90,10620600.19,10629847.17,10639137.83,10648472.33,10657850.83,10667273.33,10676739.97,10686250.97,10695806.30


In [10]:
# Distribute the future total vehicle count to counties
# based on the current percentage of vehicles in each county
# NOTE:Only consider LDV for now
current_total_count = ldv_proj_df[ldv_proj_df['Vehicle Type'] == 'Total']

# Create dataframe for future vehicle count by county
vehicle_by_county_proj = pd.DataFrame()

for year in year_cols:
    vehicle_by_county_proj[year] = vehicle_by_county['Percentage'] * current_total_count[year].to_numpy()
    vehicle_by_county_proj[year] = vehicle_by_county_proj[year].astype(int)

vehicle_by_county_proj['County'] = vehicle_by_county['County']
vehicle_by_county_proj = vehicle_by_county_proj.set_index('County')
vehicle_by_county_proj


,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,...,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050
County,,,,,,,,,,,,,,,,,,,,,
Albany County,214923,215095,215267,215441,215615,215791,215967,216144,216322,216501,...,218715,218906,219097,219289,219482,219676,219871,220067,220263,220461
Allegany County,33515,33542,33569,33596,33623,33651,33678,33706,33734,33761,...,34107,34136,34166,34196,34226,34257,34287,34317,34348,34379
Bronx County,268913,269128,269344,269562,269780,269999,270220,270441,270664,270888,...,273659,273897,274136,274376,274618,274861,275104,275349,275595,275843
Broome County,134007,134115,134222,134331,134439,134549,134659,134769,134880,134991,...,136372,136491,136610,136730,136850,136971,137093,137215,137337,137460
Cattaraugus County,54841,54885,54929,54973,55018,55062,55107,55152,55198,55243,...,55809,55857,55906,55955,56004,56054,56103,56153,56204,56254
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Washington County,52901,52943,52986,53029,53071,53115,53158,53202,53245,53289,...,53834,53881,53928,53976,54023,54071,54119,54167,54215,54264
Wayne County,79637,79701,79765,79829,79894,79959,80024,80090,80156,80222,...,81043,81113,81184,81255,81327,81399,81471,81543,81616,81689
Westchester County,622778,623276,623776,624279,624785,625293,625804,626317,626832,627351,...,633767,634319,634873,635430,635989,636551,637115,637683,638253,638825


In [11]:
# Save the result to csv file
vehicle_by_county_proj.to_csv(os.path.join(output_dir, 'vehicle_by_county_proj.csv'))